<p>Example showing how to load and process the manual annotations</p>
<p>We show how to load the training, validation, and test paritions. As well, we show how to load the test data for the 3 annotators.</p>

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../skin3d')))
from skin3d.bodytex import BodyTexDataset

In [3]:
# File path of the bodytex CSV.
bodytex_csv = '../data/3dbodytex-1.1-highres/bodytex.csv'
bodytex_df = pd.read_csv(bodytex_csv, converters={'scan_id': lambda x: str(x)})
bodytex_df.head()
# Each row represents a mesh, with a unique `scan_id` and `scan_name`.
# We use the `scan_id` to identify each mesh.
# The `scan_name` corresponds to the folder name used by 3DBodyTex.

# Each subject has two meshes, that can be identified with the `subject_id`,
# e.g., '000-001' means mesh `000` and mesh `001` are of the same subject.

# Each subject is assigned to a partition (e.g., train, test).

# If a mesh has `selected=True`, then we provide manual annotations.
# Otherwise, we do not provide annotations for the mesh. 

,subject_id,scan_name,scan_id,sex,pose,partition,selected
0,000-001,000-f-run,000,f,run,test,True
1,000-001,001-f-u,001,f,u,test,False
2,002-003,002-f-run,002,f,run,train,True
3,002-003,003-f-u,003,f,u,train,True
4,004-005,004-f-run,004,f,run,train,True


In [4]:
bodytex = BodyTexDataset(df=bodytex_df)
bodytex.summary()

Number of scans annotated with lesions: 218
Number of annotated training scans: 128
Number of annotated validation scans: 40
Number of annotated testing scans: 40
Number of annotated longitudinal scans: 10
Total number of lesions annotated: 26507
Average annotated lesion width=22.07, height=22.95


In [5]:
# You can get a dataframe of the data used for training.
train_df = bodytex.annotated_samples_in_partition('train')
train_df.head()

,subject_id,scan_name,scan_id,sex,pose,partition,selected
2,002-003,002-f-run,002,f,run,train,True
3,002-003,003-f-u,003,f,u,train,True
4,004-005,004-f-run,004,f,run,train,True
5,004-005,005-f-u,005,f,u,train,True
6,006-007,006-f-run,006,f,run,train,True


In [6]:
# You can get a dataframe of the validation data.
valid_df = bodytex.annotated_samples_in_partition('valid').head()
valid_df.head()

,subject_id,scan_name,scan_id,sex,pose,partition,selected
10,010-011,010-f-run,010,f,run,valid,True
11,010-011,011-f-u,011,f,u,valid,True
18,018-019,018-f-scape013,018,f,scape013,valid,True
22,022-023,022-f-scape061,022,f,scape061,valid,True
30,030-031,030-f-scape059,030,f,scape059,valid,True


In [7]:
# You can get a dataframe of the test data.
test_df = bodytex.annotated_samples_in_partition('test')
test_df.head()

,subject_id,scan_name,scan_id,sex,pose,partition,selected
0,000-001,000-f-run,000,f,run,test,True
9,008-009,009-f-u,009,f,u,test,True
21,020-021,021-f-u,021,f,u,test,True
24,024-025,024-f-scape046,024,f,scape046,test,True
36,036-037,036-f-scape010,036,f,scape010,test,True


In [8]:
# You can get the bounding boxes for scan_id '002'
# that belongs to the training data.
scan_id = train_df.scan_id.values[0]
print(scan_id)
bodytex.annotation(scan_id).head()
# Each row represents a bounding box.
# x,y indicate the bounding box starting coordinate within the texture image.
# x2, y2 indicate the ending coordinate of the bounding box.
# width, height are the width and height of the bounding box with respect to x,y.

# (x2, y2) and (width, height) are equivalent ways of representing 
# the bounding box on the 2D texture image.
# x + width = x2
# y + height = y2

002


,scan_id,x,y,x2,y2,width,height,annotator,lesion_id
0,002,779,877,797,895,18,18,,
1,002,1138,827,1161,846,23,19,,
2,002,1013,902,1033,921,20,19,,
3,002,702,808,721,823,19,15,,
4,002,685,753,702,771,17,18,,


In [9]:
# Similarly, you can get the bounding boxes
# for scan_id '010' in the validation data.
scan_id = valid_df.scan_id.values[0]
print(scan_id)
bodytex.annotation(scan_id).head()

010


,scan_id,x,y,x2,y2,width,height,annotator,lesion_id
0,010,238,1165,260,1188,22,23,,
1,010,204,1066,219,1081,15,15,,
2,010,260,1067,276,1079,16,12,,
3,010,1008,616,1026,631,18,15,,
4,010,1118,641,1140,660,22,19,,


In [10]:
# For testing, we evaluated against 3 annotators, 
# where each annotator provided independent bounding boxes for each test scan.
# For scans in the "test" partition, you can specify the annotator 
# along with the scan_id to get the bounding boxes for a specific annotator.
scan_id = test_df.scan_id.values[10]
print(scan_id)
# Here's how you specify the `annotator` for scans in the test partition.
# (this won't work for scans that are not part of the "test" partition)
bodytex.annotation(scan_id, annotator='A1').head()

089


,scan_id,x,y,x2,y2,width,height,annotator,lesion_id
0,089,1287,290,1299,299,12,9,A1,
1,089,1286,301,1298,313,12,12,A1,
2,089,1207,252,1221,267,14,15,A1,
3,089,1005,232,1020,246,15,14,A1,
4,089,1307,170,1319,181,12,11,A1,


In [11]:
# Another annotator for the same test scan ID of '089'.
bodytex.annotation(scan_id, annotator='A2').head()

,scan_id,x,y,x2,y2,width,height,annotator,lesion_id
0,089,1285,301,1300,318,15,17,A2,
1,089,633,1410,657,1431,24,21,A2,
2,089,834,1418,854,1444,20,26,A2,
3,089,713,1434,736,1461,23,27,A2,
4,089,566,2134,582,2155,16,21,A2,


In [12]:
# The final annotator A3 on the test scan ID of '089'.
bodytex.annotation(scan_id, annotator='A3').head()

,scan_id,x,y,x2,y2,width,height,annotator,lesion_id
0,089,634,1407,654,1434,20,27,A3,
1,089,713,1434,734,1458,21,24,A3,
2,089,841,1425,857,1441,16,16,A3,
3,089,557,2258,580,2282,23,24,A3,
4,089,1319,2886,1342,2911,23,25,A3,


In [13]:
# Please see the other example notebooks for additional details.